In [ ]:
# Copyright 2022 Diabwellness.ai, Inc.
# All rights reserved

# Drug efficiency

Notebook to map drug names to generic names and finally to drug classes (scientific composition) and measure the efficiency of possible first-prescription drug combinations based on first two recorded HbA1c values of patients.

We use the following tables for this analysis: 
- `prescription_details`: contains the prescriptions corresponding to every appointment
- `drug_master`: contains the drug names to generic names mapping
- `measurement_details`: contains the various measurements corresponding to every appointments
- `patient_details`: contains the age, gender and diabetic duration of the patients

In [ ]:
# necessary library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from diabwellness.utils.data_utils import (
    map_generic_names,
    map_drug_classes,
    extract_drug_combinations,
    preprocess_measurements,
    a1c_aggregator,
    calculate_efficacy,
    extract_biguanide_combinations,
)

# Change jupyter notebook to full width for extra visualization space
from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option("display.max_rows", 50000)
pd.set_option("display.max_columns", None)

In [ ]:
pres_df = pd.read_csv(
    "../database/prescription_details.csv", on_bad_lines="skip", low_memory=False
)
pres_df.info()

drug_df = pd.read_csv(
    "../database/drug_master.csv", on_bad_lines="skip", low_memory=False
)
drug_df.info()

colnames = [
    "ID",
    "APPOINT_ID",
    "BMI",
    "BP",
    "COMPLAINTS",
    "CREATED_BY",
    "CREATED_DATE",
    "DIAGNOSIS",
    "HEIGHT",
    "LOCATION_ID",
    "NFID",
    "STATUS",
    "TEMPERATURE",
    "UPDATED_BY",
    "UPDATED_DATE",
    "WC",
    "WEIGHT",
    "PATIENT_TYPE",
    "A1C",
    "DIA_BP",
    "DURATION_TT",
    "FS",
    "NOTES",
    "PP",
    "PULSE",
    "REVIEW_DATYS",
    "ADMISSION_REQUIRED",
    "REVIEW_DATE",
    "LAB_FOR_NEXT_VISIT",
]
meas_df = pd.read_csv(
    "../database/measurement_details.tsv",
    sep="\t",
    names=colnames,
    header=None,
    parse_dates=["CREATED_DATE", "UPDATED_DATE"],
    low_memory=False,
)
meas_df.info()

### Takeaways for prescription details:
1. APPOINT_ID: 76.3k entries are 0; no outliers!
2. DETAIL_ID: 76.8k entries are 0; more info on this column is required. remove it.
3. DRUG_DOSE: 471.2k entries are 1; not uniform, best to remove it
4. DRUG_NAME: is mostly fine, didn't see any problems
5. DRUG_TYPE: contains 8 types; no problems; schedule it
6. INTAKE: contains NLP and tamil phrases as well; better to remove it
7. ITEM_ID: seems proper; what is it btw? cross check Drug master consultant.
8. PATIENT_ID: seems proper as well
9. PATIENT_OLD_ID: seems proper; correlate it with the new data.
10. QTY: no problems as such, but can be ambiguous when different drugs are compared, so remove it?
11. DRUG_STRENGTH: 94.7k entries are -; ambiguous; best to remove it?
12. PHARMACY_DRUG_ID: 512.2k entries are 0; remove it?
13. PHARMACY_SENT_IND: looks proper; only two categories; what does it refer to tho? remove it.

### Takeaways for drug master:
1. DRUG_NAME: Always existing; can't find outliers or invalid names as a ML engineer
2. DRUG_TYPE: Has some outliers and repeated values such as (TAB and Tab, INJ and INJ.)
3. GENERIC_NAME: not so generic; not just composition but also the measurements are there. How do we make it generic?
4. INSULIN_CONTAINS: quite random; more information required; bottle 400 catridge 300; 20 units/day
5. DOSAGE_TYPE: few outliers; more information required
6. DRUG_ID: seems proper; should be used to link with prescriptions

### Takeaways for measurement details:
1. APPOINT_ID: 116k entries are 0; in measurements analysis can be done without APPOINT ID; some appoint ID repeated -> actually the same entry repeated, so drop duplicates!
   Take Created Date as input to link and create artificial Appoint ID for 0 IDs.
2. NFID: Very clean; nothing to change
3. COMPLAINTS: The nil cases include some descriptions as well sometimes; and nil itself is valuable! 
   Perhaps fillna with "nil"? Okay
   there are some "Nan" strings as well; okay
   also this is where COMPLAINTS, PATIETNT_TYPE and DIAGNOSIS are all "Nan" (6235 entries). What do we do about them?
4. PATIENT_TYPE: 14k Nan values; 6236 "Nan" strings; outliers can be processed to convert into good values; get help from doctor; 
   relevant entries: DM, NON DM, THY, HT, NON MS, IDDM, InflDisorder 
   Impute same patient type for a patient; still Nan: impute values from Complaints clues;
   "30", "15": replace with DM
    NON DM: Allergy, Anemia, Hlip, Hyperlipidemia
    GDM: DM
    CKD: based on diagnosis
5. DIAGNOSIS: Check Nan values, is there something that we can do about this? Ask doctor; perhaps intented values?
6. HEIGHT: 2887 entries are 0; some outliers; zeros can be filled by the mean/mode for the same patient
7. WEIGHT: 2384 entries are 0; some outliers; zeros can be filled by mean/ mode for the same patient (this actually can be problematic, b/c patients can increase their weights); interpolation between values?
8. BMI: recompute this entirely based on height and weight
9. A1C: 57.5k entries are 0; could be because of infrequent recording; some text fields, so convert to numeric and coerce; some outliers; 
   infrequent information recording; like a reel; 3-4 months once recorded.
   range = (4, 24) double digits maybe missing a decimal point!
10. BP: 10.7k records are 0; some outliers and texts; recorded once in three months
    range = (40, 200)
11. DIA_BP: 9.9k entries are 0; some outliers; recorded once in three months
    range = (40, 160)
12. FS: 40k entries are 0; some outliers
    range = (15, 1000)
13. PP: 25.1k entries are 0; some outliers
    range = (30, 1000)
14. PULSE: 9316 entires are 0; some outliers and texts
    range = (40, 160)    

## Extracting drug combinations:

In [ ]:
pres_df = map_generic_names(pres_df, drug_df)
pres_df = map_drug_classes(pres_df)
pres_df.head()

In [ ]:
pres_filt_df = extract_drug_combinations(pres_df)
pres_filt_df.head()

In [ ]:
# visualize the frequency of each first-prescriotion drug combination:
pres_filt_df["DRUG_COMBINATION"].value_counts()

In [ ]:
ax = pres_filt_df["DRUG_COMBINATION"].value_counts().plot(kind="bar", figsize=(10, 8))
ax.tick_params(
    axis="x",  # changes apply to the x-axis
    which="both",  # both major and minor ticks are affected
    bottom=False,  # ticks along the bottom edge are off
    top=False,  # ticks along the top edge are off
    labelbottom=False,
)  # labels along the bottom edge are off

## Extracting A1c data:

In [ ]:
meas_df = preprocess_measurements(meas_df)
meas_df.head()

In [ ]:
meas_filt_df = meas_df.groupby(["NFID"]).apply(a1c_aggregator).dropna()
meas_filt_df.head()

In [ ]:
meas_filt_df.info()
# total number of new DM patients
print("New T2DM patients: ", meas_filt_df.NEW_T2DM.sum())

## Calculating drug efficacy:

In [ ]:
# combine the drug combinations and A1C values, count the drugs:
efficacy_df = calculate_efficacy(pres_filt_df, meas_filt_df)
efficacy_df.head()

In [ ]:
efficacy_df.mean_a1c_second.plot.hist(bins=50)

In [ ]:
# sort according to p-values:
counts_cond = efficacy_df["patient_counts"] > 80
efficacy_df.loc[counts_cond].sort_values(["p_value"], ascending=True)
# sort according to mean a1c reduction:
# efficacy_df.loc[counts_cond].sort_values(["mean_a1c_reduction"], ascending=True)

In [ ]:
# plot the z-statistics:
ax = (
    efficacy_df.loc[counts_cond]
    .sort_values(["p_value"], ascending=True)["z_statistics"]
    .plot.bar(figsize=(15, 10))
)
plt.xticks(ha="right", rotation=45, fontsize=15, fontname="monospace")
plt.yticks(fontsize=15, fontname="monospace")
plt.xlabel("Drug combinations", fontsize=15, fontname="monospace")
plt.ylabel("z-statistics", fontsize=15, fontname="monospace")
plt.show()

In [ ]:
extract_biguanide_combinations(efficacy_df, drug_counts=2)

In [ ]:
extract_biguanide_combinations(efficacy_df, drug_counts=3)

In [ ]:
extract_biguanide_combinations(efficacy_df, drug_counts=4)